# Files and Interchage Formats
---
## Files

file_object = `open(`file_name`, mode='r')`

|Mode	| Meaning           |
|:---------:|:------------------|
|`'r'`	| open for reading (default)|
|`'w'`	| open for writing, truncating the file first |
|`'x'`	| open for exclusive creation, failing if the file already exists |
|`'a'`	| open for writing, appending to the end of the file if it exists |
|`'b'`	| binary mode |
|`'t'`	| text mode (default)|
|`'+'`	| open a disk file for updating (reading and writing) |

`sys.stdin, sys.stdout, sys.error` file object는 이미 open되어 있다.

### Writing into files

In [ ]:
filename = "test.txt"
myfile = open(filename, "w")
myfile.write("My first file written from Python\n")
myfile.write("---------------------------------\n")
myfile.write("지구야, 안녕!\n")
myfile.close()

### Reading from text files
Read all at a time

In [ ]:
file = open(filename)
content = file.read()
file.close()
content

Read a file into a list of lines

In [ ]:
file = open(filename)
content = file.readlines()
file.close()
content

Read a line at a time

In [ ]:
file = open(filename)
while True:
    line = file.readline()
    if line == '':     # empty str if eof reached
        break
    print(line, end='')
file.close()

Using `for` loop

In [ ]:
file = open(filename)
for line in file.readlines():
    print(line, end='')
file.close()

### File object is **iterable**!!
`for` loop에서 한 라인씩 가져온다. 매 iteration 마다 `file.readline()`이 실행된다.
> 라인 마다 disk file을 access해야기 때문에 성능이 저하될 거라 우려할 필요없다. Buffering되어 있기 때문에 `readline()` method는 거의 memory만 access할 것이다.

In [ ]:
file = open(filename)
for line in file:
    print(line, end='')
file.close()

> 위 code는 correct한가? </br>
> file이 존재하지 않거나 read할 수 없으면 error!!
### 가급적 with statement를 사용하자!!
`with` statement는 context manager다. 들어갈 때와 나올 때 필요한 조치를 `with` 절에 주어진 context에 따라 적절한 action을 자동으로 실행한다.

File object에 대해서는 open할 때 error가 발생하면 exception handling 해주고, 나갈 때는 closing과 기타 cleaning action을 취한다.

In [ ]:
with open(filename) as file:
    for line in file:
        print(line, end='')

In [ ]:
file.closed     # check if closed

### Working with binary files
- Binary mode로 open
- read하면 `str`이 아니라 `bytes` type object으로 return된다.
- write할 때, `str`이 아니라 `bytes` type object을 줘야 한다.

예: copy file
```Python
f = open("somefile.zip", "rb")
g = open("thecopy.zip", "wb")

while True:
    buf = f.read(1024)   # read up to 1024 bytes
    if len(buf) == 0:
         break
    g.write(buf)

f.close()
g.close()
```

## Python string은 unicode표현된다.
전 세계 문자를 unique하게 표현하고 프로그램에서 사용한다. (코드가 검색, 비교 가능하다)
- encode: 파일에 저장하거나 Internet에서 통신할 때, `bytes` type으로 바꿔야 한다.
  - Unix: utf-8 encoding
  - Windows: ms949
- decode: 파일에서 읽거나 Internet에서 도착한 문자는 unicode로 바꾼후 processing해야 한다.

In [ ]:
hs = 'Hello. 안녕하세요.'
for code in ('utf-8', 'ms949'):
    encode = hs.encode(code)
    print(f'{code}:', encode)
    print('unicode:', encode.decode(code))

## Text file 이해하기
Printable character들로 이루어지며 line ending(termination) 문자가 있다.

Line ending이 OS에 따라 다르다:
- Unix, Mac OS X: `\n`
- Windows: `\r\n`
- Mac OS X 이전 Mac: `\r`

Disk file encoding: byte 단위로 표현되도록 바꿔줘야 한다.
- Unix: utf-8 encoding
- Windows: ms949

In [ ]:
with open(filename, "rb") as f:
    content = f.read()
print(content)
content.decode('ms949')  # on Windows
# content.decode('utf-8') # on UNIX

## Saving structured data with JSON format
JSON(JavaScript Object Notation)은 nested list나 dictionary 등 복잡한 data type을 serialize하는 널리 쓰이는 data interchange format이다. (language/platform independent)

> 복잡한 data structure를 저장하거나 network을 통해 송신하려면, serialize해야 한다. 널리 이용되는 JSON format으로 바꿔 저장하거나 통신해라.

#### Convert Python data into json string

In [ ]:
import json
data = {'pears': 217, 'apples': 430, 'oranges': 525, 'bananas': 312}
s = json.dumps(data)
s

#### Saving into JSON format text file

In [ ]:
with open('serialized.txt', 'w') as f:
    json.dump(data, f)

#### Loading the JSON file

In [ ]:
with open('serialized.txt') as f:
    inventory = json.load(f)
inventory

## `pickle` to store native Python objects

- Support persistent objects: 대부분의 Python object들을 저장할 수 있다. Class object도 저장 가능하다. 
- 하지만, 다른 언어로 작성된 application과 호환성이 없다. 

In [ ]:
import pickle

# An arbitrary collection of objects supported by pickle.
data = {
    'a': [1, 2.0, 3, 4+6j],
    'b': ("character string", b"byte string"),
    'c': {None, True, False}
}

with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)
data